In [1]:
import os
import pandas as pd
from nba_api.stats.static import teams
from pathlib import Path

# Setup data directory path - go up one level from notebooks to project root
BASE_DIR = Path(os.getcwd()).parent
DATA_DIR = BASE_DIR / 'data' / 'game3'
os.makedirs(DATA_DIR, exist_ok=True)

print(f"Working directory: {BASE_DIR}")
print(f"Data directory: {DATA_DIR}")
print(f"Data directory exists: {DATA_DIR.exists()}")
print(f"Data directory is writable: {os.access(DATA_DIR, os.W_OK)}")

# Get team IDs
OKC_data = teams.find_teams_by_full_name("Oklahoma City Thunder")
OKC_id = OKC_data[0]['id'] # 1610612760

PAC_data = teams.find_teams_by_full_name("Indiana Pacers")
PAC_id = PAC_data[0]['id'] # 1610612754


Working directory: /Users/medhasubramaniyan/2025_finals
Data directory: /Users/medhasubramaniyan/2025_finals/data/game3
Data directory exists: True
Data directory is writable: True


In [2]:
from nba_api.stats.endpoints import leaguegamefinder

finder = leaguegamefinder.LeagueGameFinder(
    player_or_team_abbreviation="T",
    team_id_nullable=OKC_id,
    date_from_nullable="06/11/2025",
    date_to_nullable="06/11/2025"
)

g3 = finder.get_data_frames()[0]
g3_id = g3["GAME_ID"].iloc[0] # Should be 0042400403

print("Game Data:")
print(g3)


Game Data:
  SEASON_ID     TEAM_ID TEAM_ABBREVIATION              TEAM_NAME     GAME_ID  \
0     42024  1610612760               OKC  Oklahoma City Thunder  0042400403   

    GAME_DATE    MATCHUP WL  MIN  PTS  ...  FT_PCT  OREB  DREB  REB  AST  STL  \
0  2025-06-11  OKC @ IND  L  239  107  ...   0.767     9    33   42   16    6   

   BLK  TOV  PF  PLUS_MINUS  
0    4   17  20        -9.0  

[1 rows x 28 columns]


In [3]:
# Traditional Stats
from nba_api.stats.endpoints import boxscoretraditionalv2
box = boxscoretraditionalv2.BoxScoreTraditionalV2(game_id=g3_id)

# Player Stats
players = box.player_stats.get_data_frame()
player_path = DATA_DIR / 'player_data.csv'
players.to_csv(player_path)
print(f"Player data saved to: {player_path}")

# Team Stats
teams = box.team_stats.get_data_frame()
team_path = DATA_DIR / 'team_data.csv'
teams.to_csv(team_path)
print(f"Team data saved to: {team_path}")


Player data saved to: /Users/medhasubramaniyan/2025_finals/data/game3/player_data.csv
Team data saved to: /Users/medhasubramaniyan/2025_finals/data/game3/team_data.csv


In [4]:
# Advanced Stats
from nba_api.stats.endpoints import boxscoreadvancedv2
adv_box = boxscoreadvancedv2.BoxScoreAdvancedV2(game_id=g3_id)

# Advanced Player Stats
adv_player_df = adv_box.player_stats.get_data_frame()
adv_player_path = DATA_DIR / 'adv_players.csv'
adv_player_df.to_csv(adv_player_path)
print(f"Advanced player data saved to: {adv_player_path}")

# Advanced Team Stats
adv_team_df = adv_box.team_stats.get_data_frame()
adv_team_path = DATA_DIR / 'adv_team.csv'
adv_team_df.to_csv(adv_team_path)
print(f"Advanced team data saved to: {adv_team_path}")


Advanced player data saved to: /Users/medhasubramaniyan/2025_finals/data/game3/adv_players.csv
Advanced team data saved to: /Users/medhasubramaniyan/2025_finals/data/game3/adv_team.csv


In [5]:
# Play-by-Play Data
from nba_api.stats.endpoints import playbyplayv2
pbp = playbyplayv2.PlayByPlayV2(game_id=g3_id)
pbp_df = pbp.get_data_frames()[0]
pbp_path = DATA_DIR / 'plays.csv'
pbp_df.to_csv(pbp_path)
print(f"Play-by-play data saved to: {pbp_path}")

# Hustle Stats
from nba_api.stats.endpoints import BoxScoreHustleV2
hustle = BoxScoreHustleV2(game_id=g3_id)
hustle_df = hustle.get_data_frames()[0]
hustle_path = DATA_DIR / 'hustle.csv'
hustle_df.to_csv(hustle_path)
print(f"Hustle stats saved to: {hustle_path}")

# Win Probability
from nba_api.stats.endpoints import WinProbabilityPBP
w_pct_pbp = WinProbabilityPBP(game_id=g3_id, run_type="each second")
w_pct_pbp_df = w_pct_pbp.get_data_frames()[0]
win_prob_path = DATA_DIR / 'win_prob.csv'
w_pct_pbp_df.to_csv(win_prob_path)
print(f"Win probability data saved to: {win_prob_path}")


Play-by-play data saved to: /Users/medhasubramaniyan/2025_finals/data/game3/plays.csv
Hustle stats saved to: /Users/medhasubramaniyan/2025_finals/data/game3/hustle.csv
Win probability data saved to: /Users/medhasubramaniyan/2025_finals/data/game3/win_prob.csv


In [6]:
# Four Factors
from nba_api.stats.endpoints import boxscorefourfactorsv2
four_factors = boxscorefourfactorsv2.BoxScoreFourFactorsV2(
    game_id=g3_id,
    start_period=1,
    end_period=4,
    range_type=0,
    start_range=0,
    end_range=0
)

players_four_factors, teams_four_factors = four_factors.get_data_frames()
players_ff_path = DATA_DIR / 'players_four_factors.csv'
teams_ff_path = DATA_DIR / 'teams_four_factors.csv'

players_four_factors.to_csv(players_ff_path)
teams_four_factors.to_csv(teams_ff_path)
print(f"Four factors data saved to: {players_ff_path} and {teams_ff_path}")


Four factors data saved to: /Users/medhasubramaniyan/2025_finals/data/game3/players_four_factors.csv and /Users/medhasubramaniyan/2025_finals/data/game3/teams_four_factors.csv


In [7]:
# Shot Charts
from nba_api.stats.endpoints import CommonTeamRoster, ShotChartDetail

team_id = PAC_id
season = '2024-25'
season_type = 'Playoffs'
game_id = g3_id

# 1. Fetch the team's roster for the season
roster = CommonTeamRoster(team_id=team_id, season=season)
roster_df = roster.get_data_frames()[0]
player_ids = roster_df['PLAYER_ID'].tolist()

# 2. Loop through each player and pull their shot chart detail for that game
shots_list = []
for pid in player_ids:
    shot_detail = ShotChartDetail(
        context_measure_simple='PTS',
        last_n_games=0,
        league_id='00',
        season_type_all_star=season_type,
        season_nullable=season,
        team_id=0,
        player_id=pid,
        game_id_nullable=game_id,
        date_from_nullable='06/11/2025',
        date_to_nullable='06/11/2025'
    )
    _, df_shots = shot_detail.get_data_frames()
    if not df_shots.empty:
        shots_list.append(df_shots)

if shots_list:
    team_shot_detail_df = pd.concat(shots_list, ignore_index=True)
    shot_chart_path = DATA_DIR / 'Pacers_shotchart.csv'
    team_shot_detail_df.to_csv(shot_chart_path)
    print(f"Shot chart data saved to: {shot_chart_path}")
else:
    print("No shot chart data available")


Shot chart data saved to: /Users/medhasubramaniyan/2025_finals/data/game3/Pacers_shotchart.csv
